## 一、2002 年-2018 年上海机动车拍卖

In [210]:
import numpy as np
import pandas as pd 
auction = pd.read_csv('data/2002年-2018年上海机动车拍照拍卖.csv')
auction.head()

,Date,Total number of license issued,lowest price,avg price,Total number of applicants
0,2-Jan,1400,13600,14735,3718
1,2-Feb,1800,13100,14057,4590
2,2-Mar,2000,14300,14662,5190
3,2-Apr,2300,16000,16334,4806
4,2-May,2350,17800,18357,4665


In [5]:
auction.shape

(203, 5)

In [6]:
auction.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 203 entries, 0 to 202
Data columns (total 5 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   Date                            203 non-null    object
 1   Total number of license issued  203 non-null    int64 
 2   lowest price                    203 non-null    int64 
 3   avg price                       203 non-null    int64 
 4   Total number of applicants      203 non-null    int64 
dtypes: int64(4), object(1)
memory usage: 8.1+ KB


### (1) 哪一次拍卖的中标率首次小于 5%？

In [34]:
auction['gain_rate'] = (auction['Total number of license issued']/auction['Total number of applicants'])

In [38]:
auction.query('gain_rate < 0.05').head(1)

,Date,Total number of license issued,lowest price,avg price,Total number of applicants,gain_rate
159,15-May,7482,79000,79099,156007,0.047959


### (2) 按年统计拍卖最低价的下列统计量：最大值、均值、0.75 分位数，要求显示在同一张表上。

In [39]:
auction = pd.read_csv('data/2002年-2018年上海机动车拍照拍卖.csv')

In [42]:
auction.head()

,Date,Total number of license issued,lowest price,avg price,Total number of applicants
0,2-Jan,1400,13600,14735,3718
1,2-Feb,1800,13100,14057,4590
2,2-Mar,2000,14300,14662,5190
3,2-Apr,2300,16000,16334,4806
4,2-May,2350,17800,18357,4665


In [78]:
date_split = auction['Date'].str.split('-',expand=True)

In [80]:
year_list = date_split[0].to_list()

In [86]:
for i in range(len(year_list)):
    if len(year_list[i]) == 1:
        year_list[i] = '200' + year_list[i]
    if len(year_list[i]) == 2:
        year_list[i] = '20' + year_list[i]

In [90]:
date_split[0] = year_list

In [100]:
auction['Year'] = date_split[0]
auction['Month'] =date_split[1]

In [103]:
auction_new = auction.drop(columns='Date')

In [128]:
auction_group = auction_new.groupby('Year')['lowest price '].agg(['max','mean']).round(2)

In [137]:
q_75 = auction_new.groupby('Year')['lowest price '].quantile(q=.75)
q_75.name ='q_75'

In [166]:
auction_group.join(q_75).head()

,max,mean,q_75
Year,,,
2002,30800,20316.67,24300.0
2003,38500,31983.33,36300.0
2004,44200,29408.33,38400.0
2005,37900,31908.33,35600.0
2006,39900,37058.33,39525.0


### (3) 将第一列时间列拆分成两个列，一列为年份（格式为 20××），另一列为月份（英语缩写），添加到列表作为第一第二列，并将原表第一列删除，其他列依次向后顺延。

In [165]:
auction_new.set_index(['Year','Month']).reset_index('Month').head()

,Month,Total number of license issued,lowest price,avg price,Total number of applicants
Year,,,,,
2002,Jan,1400,13600,14735,3718
2002,Feb,1800,13100,14057,4590
2002,Mar,2000,14300,14662,5190
2002,Apr,2300,16000,16334,4806
2002,May,2350,17800,18357,4665


### (4) 现在将表格行索引设为多级索引，外层为年份，内层为原表格第二至第五列的变量名，列索引为月份。

In [159]:
auction_m = auction_new.set_index(['Year','Total number of license issued', \
                                   'lowest price ', 'avg price',
                                   'Total number of applicants'])

In [164]:
auction_m.head()

Month
Year Total number of license issued lowest price  avg price Total number of applicants      
2002 1400                           13600         14735     3718                         Jan
     1800                           13100         14057     4590                         Feb
     2000                           14300         14662     5190                         Mar
     2300                           16000         16334     4806                         Apr
     2350                           17800         18357     4665                         May

### (5) 一般而言某个月最低价与上月最低价的差额，会与该月均值与上月均值的差额具有相同的正负号，哪些拍卖时间不具有这个特点？

In [222]:
s_low = auction['lowest price '].copy()

In [223]:
s_low.index = range(1,204)

In [224]:
lowest_price = auction['lowest price '] - s_low

In [226]:
s_avg = auction['avg price'].copy()
s_avg.index = range(1,204)
avg_price = auction['avg price'] - s_avg

In [250]:
auction[( lowest_price * avg_price ).iloc[:-1] < 0]['Date']

21      3-Oct
22      3-Nov
29      4-Jun
36      5-Jan
37      5-Feb
44      5-Sep
52      6-May
56      6-Sep
60      7-Jan
61      7-Feb
71      7-Dec
128    12-Oct
Name: Date, dtype: object

### (6) 将某一个月牌照发行量与其前两个月发行量均值的差额定义为发行增益，最初的两个月用 0 填充，求发行增益极值出现的时间。

In [251]:
auction.head()

,Date,Total number of license issued,lowest price,avg price,Total number of applicants
0,2-Jan,1400,13600,14735,3718
1,2-Feb,1800,13100,14057,4590
2,2-Mar,2000,14300,14662,5190
3,2-Apr,2300,16000,16334,4806
4,2-May,2350,17800,18357,4665


In [252]:
issue_number = auction['Total number of license issued'].copy()

In [268]:
issue_number.index = range(1,204)
mean = ( issue_number + auction['Total number of license issued'] ) / 2
mean.index = range(1,205)
issue_add = ( auction['Total number of license issued'] - mean ).iloc[:-2].fillna(0)

In [282]:
auction['issue_add'] = issue_add

In [288]:
auction.set_index('Date')['issue_add'].idxmax()

'8-Jan'

In [289]:
auction.set_index('Date')['issue_add'].idxmin()

'8-Apr'

## 二、2007 年-2019 年俄罗斯机场货运航班运载量

In [291]:
air = pd.read_csv('data/2007年-2019年俄罗斯货运航班运载量.csv')
air.head()

,Airport name,Year,January,February,March,April,May,June,July,August,September,October,November,December,Whole year,Airport coordinates
0,Abakan,2019,44.70,66.21,72.7,75.82,100.34,78.38,63.88,73.06,66.74,75.44,110.5,89.8,917.57,"(Decimal('91.399735'), Decimal('53.751351'))"
1,Aikhal,2019,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.00,"(Decimal('111.543324'), Decimal('65.957161'))"
2,Loss,2019,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.00,"(Decimal('125.398355'), Decimal('58.602489'))"
3,Amderma,2019,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.00,"(Decimal('61.577429'), Decimal('69.759076'))"
4,Anadyr (Carbon),2019,81.63,143.01,260.9,304.36,122.00,106.87,84.99,130.00,102.00,118.00,94.0,199.0,1746.76,"(Decimal('177.738273'), Decimal('64.713433'))"


In [292]:
air.shape

(3711, 16)

In [293]:
air.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3711 entries, 0 to 3710
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Airport name         3711 non-null   object 
 1   Year                 3711 non-null   int64  
 2   January              3711 non-null   float64
 3   February             3711 non-null   float64
 4   March                3711 non-null   float64
 5   April                3711 non-null   float64
 6   May                  3711 non-null   float64
 7   June                 3711 non-null   float64
 8   July                 3711 non-null   float64
 9   August               3711 non-null   float64
 10  September            3711 non-null   float64
 11  October              3711 non-null   float64
 12  November             3711 non-null   float64
 13  December             3711 non-null   float64
 14  Whole year           3711 non-null   float64
 15  Airport coordinates  3711 non-null   o

### (1) 求每年货运航班总运量。

In [299]:
air.groupby('Year')['Whole year'].agg('sum')

Year
2007    659438.23
2008    664682.46
2009    560809.77
2010    693033.98
2011    818691.71
2012    846388.03
2013    792337.08
2014    729457.12
2015    630208.97
2016    679370.15
2017    773662.28
2018    767095.28
2019    764606.27
Name: Whole year, dtype: float64

### (2) 每年记录的机场都是相同的吗？

In [308]:
air.groupby('Year')['Airport name'].agg('count')

Year
2007    292
2008    292
2009    292
2010    292
2011    292
2012    292
2013    292
2014    292
2015    292
2016    292
2017    292
2018    248
2019    251
Name: Airport name, dtype: int64

In [309]:
air.groupby('Airport name')['Year'].agg('count')

Airport name
Abakan           13
Achinsk          11
Aikhal           13
Amderma          13
Anadyr           12
                 ..
Лешуконское      13
Мотыгино         13
Нюрба            13
Среднеколымск    13
Таксимо          13
Name: Year, Length: 297, dtype: int64

### (3) 按年计算 2010 年-2015 年全年货运量记录为 0 的机场航班比例。

In [479]:
air_0 = air.set_index('Year').sort_index().loc[2010:2015]['Whole year']
( air_0 == 0 ).groupby('Year').agg('mean')

Year
2010    0.767123
2011    0.770548
2012    0.770548
2013    0.770548
2014    0.770548
2015    0.770548
Name: Whole year, dtype: float64

### (4) 若某机场至少存在 5 年或以上满足所有月运量记录都为 0，则将其所有年份的记录信息从表中删除，并返回处理后的表格

In [497]:
air_new = air.set_index(['Airport name','Year']).sort_index()['Whole year']
( air_new == 0 ).groupby('Airport name').agg('mean')

Airport name
Abakan           0.0
Achinsk          1.0
Aikhal           1.0
Amderma          1.0
Anadyr           0.0
                ... 
Лешуконское      1.0
Мотыгино         1.0
Нюрба            1.0
Среднеколымск    1.0
Таксимо          1.0
Name: Whole year, Length: 297, dtype: float64

## 三、新冠肺炎在美国的传播

In [500]:
death = pd.read_csv('data/美国死亡数.csv')
death.head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,2020/4/17,2020/4/18,2020/4/19,2020/4/20,2020/4/21,2020/4/22,2020/4/23,2020/4/24,2020/4/25,2020/4/26
0,84001001,US,USA,840,1001,Autauga,Alabama,US,32.539527,-86.644082,...,2,2,2,1,1,2,2,2,2,2
1,84001003,US,USA,840,1003,Baldwin,Alabama,US,30.727750,-87.722071,...,2,2,2,3,3,3,3,3,3,3
2,84001005,US,USA,840,1005,Barbour,Alabama,US,31.868263,-85.387129,...,0,0,0,0,0,0,0,0,0,0
3,84001007,US,USA,840,1007,Bibb,Alabama,US,32.996421,-87.125115,...,0,0,0,0,0,0,0,0,0,0
4,84001009,US,USA,840,1009,Blount,Alabama,US,33.982109,-86.567906,...,0,0,0,0,0,0,0,0,0,0


In [502]:
death.shape

(3142, 108)

In [503]:
death.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3142 entries, 0 to 3141
Columns: 108 entries, UID to 2020/4/26
dtypes: float64(2), int64(100), object(6)
memory usage: 2.6+ MB


### (1) 用 corr() 函数计算县（每行都是一个县）人口与表中最后一天记录日期死亡数的相关系数。


### (2) 截止到 4 月 1 日，统计每个州零感染县的比例。

In [508]:
diagnosis_confimed = pd.read_csv('data/美国确证数.csv')
diagnosis_confimed.head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,2020/4/17,2020/4/18,2020/4/19,2020/4/20,2020/4/21,2020/4/22,2020/4/23,2020/4/24,2020/4/25,2020/4/26
0,84001001,US,USA,840,1001,Autauga,Alabama,US,32.539527,-86.644082,...,26,25,26,28,30,32,33,36,36,37
1,84001003,US,USA,840,1003,Baldwin,Alabama,US,30.727750,-87.722071,...,103,109,112,117,123,132,143,147,147,161
2,84001005,US,USA,840,1005,Barbour,Alabama,US,31.868263,-85.387129,...,15,18,20,22,28,29,30,32,32,33
3,84001007,US,USA,840,1007,Bibb,Alabama,US,32.996421,-87.125115,...,24,26,28,32,32,34,33,34,34,38
4,84001009,US,USA,840,1009,Blount,Alabama,US,33.982109,-86.567906,...,20,20,21,22,26,29,31,31,31,34


In [510]:
diagnosis_confimed.shape

(3142, 107)

In [511]:
diagnosis_confimed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3142 entries, 0 to 3141
Columns: 107 entries, UID to 2020/4/26
dtypes: float64(2), int64(99), object(6)
memory usage: 2.6+ MB


In [554]:
diagnosis = diagnosis_confimed[['Admin2','Province_State','2020/4/1']]\
.set_index('Province_State')

In [568]:
(diagnosis['2020/4/1'] == 0).groupby('Province_State').agg('mean')

Province_State
Alabama                 0.119403
Alaska                  0.793103
Arizona                 0.000000
Arkansas                0.293333
California              0.137931
Colorado                0.218750
Connecticut             0.000000
Delaware                0.000000
District of Columbia    0.000000
Florida                 0.164179
Georgia                 0.125786
Hawaii                  0.200000
Idaho                   0.386364
Illinois                0.480392
Indiana                 0.108696
Iowa                    0.404040
Kansas                  0.609524
Kentucky                0.441667
Louisiana               0.062500
Maine                   0.250000
Maryland                0.041667
Massachusetts           0.142857
Michigan                0.192771
Minnesota               0.367816
Mississippi             0.060976
Missouri                0.391304
Montana                 0.625000
Nebraska                0.752688
Nevada                  0.470588
New Hampshire           0.10

### (3)请找出最早出确证病例的三个县。

In [660]:
for columns in diagnosis_confimed.columns[11:]:
    top = diagnosis_confimed[diagnosis_confimed[columns]> 0]['Admin2'].values
    for i in range(len(top)):
        print(top[i])

King
King
Cook
King
Cook
King
Maricopa
Los Angeles
Orange
Cook
King
Maricopa
Los Angeles
Orange
Cook
King
Maricopa
Los Angeles
Orange
Cook
King
Maricopa
Los Angeles
Orange
Cook
King
Maricopa
Los Angeles
Orange
Cook
King
Maricopa
Los Angeles
Orange
Santa Clara
Cook
King
Maricopa
Los Angeles
Orange
Santa Clara
Cook
Suffolk
King
Maricopa
Los Angeles
Orange
Santa Clara
Cook
Suffolk
King
Maricopa
Los Angeles
Orange
San Benito
Santa Clara
Cook
Suffolk
King
Maricopa
Los Angeles
Orange
San Benito
Santa Clara
Cook
Suffolk
King
Maricopa
Los Angeles
Orange
San Benito
Santa Clara
Cook
Suffolk
King
Maricopa
Los Angeles
Orange
San Benito
Santa Clara
Cook
Suffolk
King
Maricopa
Los Angeles
Orange
San Benito
Santa Clara
Cook
Suffolk
King
Maricopa
Los Angeles
Orange
San Benito
Santa Clara
Cook
Suffolk
King
Maricopa
Los Angeles
Orange
San Benito
Santa Clara
Cook
Suffolk
King
Maricopa
Los Angeles
Orange
San Benito
Santa Clara
Cook
Suffolk
King
Maricopa
Los Angeles
Orange
San Benito
San Diego
Santa Clara
C

Jefferson
King
Kitsap
Kittitas
Pierce
Skagit
Snohomish
Thurston
Whatcom
Dane
Fond du Lac
Pierce
Waukesha
Sheridan
Jefferson
Montgomery
Maricopa
Pima
Pinal
Alameda
Calaveras
Contra Costa
Fresno
Humboldt
Imperial
Los Angeles
Madera
Marin
Orange
Placer
Riverside
Sacramento
San Benito
San Diego
San Francisco
San Joaquin
San Mateo
Santa Clara
Santa Cruz
Shasta
Solano
Sonoma
Stanislaus
Tulare
Ventura
Yolo
Adams
Arapahoe
Boulder
Denver
Douglas
Eagle
El Paso
Gunnison
Jefferson
Larimer
Pitkin
Pueblo
Routt
Summit
Weld
Fairfield
Litchfield
New Castle
Alachua
Broward
Charlotte
Clay
Collier
Duval
Hillsborough
Lee
Manatee
Miami-Dade
Nassau
Okaloosa
Palm Beach
Pasco
Pinellas
St. Johns
Santa Rosa
Sarasota
Seminole
Volusia
Bartow
Camden
Charlton
Cherokee
Cobb
Coweta
DeKalb
Fayette
Floyd
Fulton
Gordon
Gwinnett
Lee
Lowndes
Polk
Honolulu
Cook
Kane
Lake
McHenry
Adams
Boone
Hancock
Hendricks
Howard
Johnson
Marion
Noble
St. Joseph
Carroll
Harrison
Johnson
Pottawattamie
Johnson
Sedgwick
Wyandotte
Fayette
Harr

Kane
Lake
McHenry
St. Clair
Sangamon
Will
Woodford
Adams
Bartholomew
Boone
Floyd
Hamilton
Hancock
Hendricks
Howard
Johnson
LaPorte
Marion
Noble
St. Joseph
Wells
Allamakee
Carroll
Dallas
Harrison
Johnson
Polk
Pottawattamie
Butler
Franklin
Johnson
Wyandotte
Bourbon
Clark
Fayette
Harrison
Jefferson
Montgomery
Nelson
Ascension
Bossier
Caddo
Jefferson
Lafourche
Orleans
St. Bernard
St. Charles
St. John the Baptist
St. Tammany
Terrebonne
Androscoggin
Cumberland
Knox
Lincoln
Anne Arundel
Baltimore
Carroll
Charles
Frederick
Harford
Howard
Montgomery
Prince George's
Talbot
Baltimore City
Barnstable
Berkshire
Bristol
Essex
Hampden
Middlesex
Norfolk
Plymouth
Suffolk
Worcester
Bay
Charlevoix
Ingham
Kent
Macomb
Monroe
Montcalm
Oakland
Ottawa
St. Clair
Washtenaw
Wayne
Anoka
Benton
Carver
Dakota
Hennepin
Olmsted
Ramsey
Renville
Stearns
Waseca
Washington
Wright
Copiah
Forrest
Hancock
Hinds
Leflore
Monroe
Pearl River
Greene
Henry
St. Louis
Gallatin
Lewis and Clark
Missoula
Silver Bow
Yellowstone
Cass
Do

Sumter
Volusia
Barrow
Bartow
Charlton
Cherokee
Clarke
Clayton
Cobb
Columbia
Coweta
DeKalb
Dougherty
Fayette
Floyd
Forsyth
Fulton
Gordon
Gwinnett
Hall
Henry
Houston
Lee
Lowndes
Newton
Paulding
Polk
Richmond
Troup
Whitfield
Honolulu
Kauai
Maui
Ada
Blaine
Madison
Teton
Champaign
Clinton
Cook
Cumberland
DuPage
Kane
Kendall
Lake
McHenry
Madison
Peoria
St. Clair
Sangamon
Whiteside
Will
Winnebago
Woodford
Adams
Bartholomew
Boone
Clark
Fayette
Floyd
Franklin
Hamilton
Hendricks
Howard
Jennings
Johnson
Lake
LaPorte
Marion
Noble
St. Joseph
Wells
Adair
Allamakee
Black Hawk
Carroll
Dallas
Harrison
Johnson
Polk
Pottawattamie
Washington
Winneshiek
Butler
Douglas
Ford
Franklin
Johnson
Leavenworth
Wyandotte
Bourbon
Clark
Fayette
Harrison
Jefferson
Lyon
Montgomery
Nelson
Ascension
Bossier
Caddo
East Baton Rouge
Jefferson
Lafourche
Orleans
St. Bernard
St. Charles
St. John the Baptist
St. Tammany
Terrebonne
Washington
Androscoggin
Cumberland
Kennebec
Lincoln
Oxford
Penobscot
York
Anne Arundel
Baltimore
Ca

Clackamas
Deschutes
Douglas
Jackson
Klamath
Lane
Linn
Marion
Multnomah
Polk
Umatilla
Washington
Yamhill
Allegheny
Beaver
Berks
Bucks
Chester
Cumberland
Delaware
Erie
Lackawanna
Lancaster
Lebanon
Lehigh
Luzerne
Monroe
Montgomery
Northampton
Philadelphia
Pike
Washington
Wayne
Westmoreland
York
Providence
Abbeville
Anderson
Beaufort
Calhoun
Charleston
Dorchester
Fairfield
Greenville
Horry
Kershaw
Lancaster
Lee
Lexington
Richland
Saluda
Spartanburg
York
Beadle
Bon Homme
Charles Mix
Davison
McCook
Minnehaha
Pennington
Campbell
Cheatham
Cumberland
Davidson
Hamilton
Jefferson
Knox
Robertson
Rutherford
Sevier
Shelby
Sullivan
Sumner
Williamson
Bell
Bexar
Bowie
Brazoria
Brazos
Brown
Collin
Comal
Crane
Dallas
Denton
DeWitt
Ellis
El Paso
Fannin
Fort Bend
Galveston
Gregg
Grimes
Harris
Hays
Jefferson
Johnson
Lavaca
Lubbock
McLennan
Matagorda
Montgomery
Potter
Rusk
Smith
Tarrant
Travis
Van Zandt
Webb
Wichita
Williamson
Cache
Davis
Salt Lake
Summit
Tooele
Utah
Wasatch
Addison
Bennington
Chittenden
Ora

...

Howard
Howell
Iron
Jackson
Jasper
Jefferson
Johnson
Lafayette
Lewis
Lincoln
Linn
Livingston
McDonald
Macon
Madison
Maries
Marion
Miller
Mississippi
Moniteau
Montgomery
Morgan
New Madrid
Newton
Nodaway
Oregon
Osage
Pemiscot
Perry
Pettis
Phelps
Pike
Platte
Polk
Pulaski
Ralls
Randolph
Ray
Reynolds
Ripley
St. Charles
St. Clair
Ste. Genevieve
St. Francois
St. Louis
Saline
Scotland
Scott
Shelby
Stoddard
Stone
Taney
Vernon
Warren
Washington
Webster
Worth
Wright
St. Louis City
Beaverhead
Big Horn
Broadwater
Carbon
Cascade
Deer Lodge
Flathead
Gallatin
Glacier
Golden Valley
Hill
Jefferson
Lake
Lewis and Clark
Liberty
Lincoln
Madison
Meagher
Missoula
Musselshell
Park
Pondera
Ravalli
Richland
Roosevelt
Silver Bow
Stillwater
Toole
Wheatland
Yellowstone
Adams
Antelope
Box Butte
Buffalo
Burt
Butler
Cass
Cherry
Cheyenne
Clay
Colfax
Cuming
Custer
Dakota
Dawson
Dixon
Dodge
Douglas
Fillmore
Franklin
Furnas
Gage
Gosper
Greeley
Hall
Hamilton
Howard
Johnson
Kearney
Kimball
Knox
Lancaster
Lincoln
Madison
Mer

Pinal
Santa Cruz
Yavapai
Yuma
Arkansas
Ashley
Baxter
Benton
Boone
Bradley
Carroll
Chicot
Clark
Clay
Cleburne
Cleveland
Columbia
Conway
Craighead
Crawford
Crittenden
Cross
Dallas
Desha
Drew
Faulkner
Franklin
Fulton
Garland
Grant
Greene
Hempstead
Hot Spring
Howard
Independence
Izard
Jackson
Jefferson
Johnson
Lafayette
Lawrence
Lee
Lincoln
Logan
Lonoke
Madison
Marion
Miller
Mississippi
Monroe
Nevada
Newton
Ouachita
Perry
Phillips
Pike
Poinsett
Polk
Pope
Prairie
Pulaski
Randolph
St. Francis
Saline
Scott
Searcy
Sebastian
Sevier
Sharp
Stone
Union
Van Buren
Washington
White
Woodruff
Yell
Alameda
Alpine
Amador
Butte
Calaveras
Colusa
Contra Costa
Del Norte
El Dorado
Fresno
Glenn
Humboldt
Imperial
Inyo
Kern
Kings
Lake
Los Angeles
Madera
Marin
Mendocino
Merced
Mono
Monterey
Napa
Nevada
Orange
Placer
Plumas
Riverside
Sacramento
San Benito
San Bernardino
San Diego
San Francisco
San Joaquin
San Luis Obispo
San Mateo
Santa Barbara
Santa Clara
Santa Cruz
Shasta
Siskiyou
Solano
Sonoma
Stanislaus
Sutter

Klamath
Lane
Lincoln
Linn
Malheur
Marion
Morrow
Multnomah
Polk
Sherman
Tillamook
Umatilla
Union
Wallowa
Wasco
Washington
Yamhill
Adams
Allegheny
Armstrong
Beaver
Bedford
Berks
Blair
Bradford
Bucks
Butler
Cambria
Cameron
Carbon
Centre
Chester
Clarion
Clearfield
Clinton
Columbia
Crawford
Cumberland
Dauphin
Delaware
Elk
Erie
Fayette
Forest
Franklin
Fulton
Greene
Huntingdon
Indiana
Jefferson
Juniata
Lackawanna
Lancaster
Lawrence
Lebanon
Lehigh
Luzerne
Lycoming
McKean
Mercer
Mifflin
Monroe
Montgomery
Montour
Northampton
Northumberland
Perry
Philadelphia
Pike
Potter
Schuylkill
Snyder
Somerset
Sullivan
Susquehanna
Tioga
Union
Venango
Warren
Washington
Wayne
Westmoreland
Wyoming
York
Bristol
Kent
Newport
Providence
Washington
Abbeville
Aiken
Allendale
Anderson
Bamberg
Barnwell
Beaufort
Berkeley
Calhoun
Charleston
Cherokee
Chester
Chesterfield
Clarendon
Colleton
Darlington
Dillon
Dorchester
Edgefield
Fairfield
Florence
Georgetown
Greenville
Greenwood
Hampton
Horry
Jasper
Kershaw
Lancaster
Laure

Winnebago
Winneshiek
Woodbury
Worth
Wright
Atchison
Barber
Barton
Bourbon
Butler
Chase
Chautauqua
Cherokee
Clark
Clay
Cloud
Coffey
Cowley
Crawford
Dickinson
Doniphan
Douglas
Edwards
Ellis
Finney
Ford
Franklin
Geary
Gove
Grant
Gray
Greenwood
Hamilton
Harvey
Haskell
Jackson
Jefferson
Jewell
Johnson
Kearny
Kiowa
Labette
Leavenworth
Linn
Lyon
McPherson
Marion
Meade
Miami
Mitchell
Montgomery
Morris
Morton
Neosho
Osage
Osborne
Ottawa
Phillips
Pottawatomie
Pratt
Reno
Republic
Rice
Riley
Rooks
Saline
Scott
Sedgwick
Seward
Shawnee
Sheridan
Sherman
Smith
Stafford
Stanton
Stevens
Sumner
Wabaunsee
Wilson
Woodson
Wyandotte
Adair
Allen
Anderson
Ballard
Barren
Bath
Boone
Bourbon
Boyd
Boyle
Bracken
Breathitt
Breckinridge
Bullitt
Butler
Caldwell
Calloway
Campbell
Carlisle
Carroll
Carter
Casey
Christian
Clark
Clay
Clinton
Crittenden
Cumberland
Daviess
Edmonson
Fayette
Fleming
Floyd
Franklin
Fulton
Gallatin
Garrard
Grant
Graves
Grayson
Green
Greenup
Hancock
Hardin
Harrison
Hart
Henderson
Henry
Hickman
Ho

Gaines
Galveston
Gillespie
Goliad
Gonzales
Gray
Grayson
Gregg
Grimes
Guadalupe
Hale
Hamilton
Hansford
Hardin
Harris
Harrison
Hartley
Hays
Hemphill
Henderson
Hidalgo
Hill
Hockley
Hood
Hopkins
Houston
Howard
Hunt
Hutchinson
Jack
Jackson
Jasper
Jefferson
Jim Hogg
Jim Wells
Johnson
Jones
Karnes
Kaufman
Kendall
Kerr
Kleberg
Knox
Lamar
Lamb
Lampasas
La Salle
Lavaca
Lee
Leon
Liberty
Limestone
Live Oak
Llano
Lubbock
Lynn
McCulloch
McLennan
Madison
Marion
Martin
Mason
Matagorda
Maverick
Medina
Midland
Milam
Mitchell
Montague
Montgomery
Moore
Morris
Motley
Nacogdoches
Navarro
Newton
Nolan
Nueces
Ochiltree
Oldham
Orange
Palo Pinto
Panola
Parker
Parmer
Pecos
Polk
Potter
Rains
Randall
Red River
Reeves
Roberts
Robertson
Rockwall
Rusk
Sabine
San Augustine
San Jacinto
San Patricio
Scurry
Shelby
Sherman
Smith
Starr
Stephens
Swisher
Tarrant
Taylor
Terry
Titus
Tom Green
Travis
Trinity
Tyler
Upshur
Uvalde
Val Verde
Van Zandt
Victoria
Walker
Waller
Washington
Webb
Wharton
Wheeler
Wichita
Wilbarger
Willacy
